<!-- Copyright (c) 2021-2024, InterDigital Communications, Inc
All rights reserved.

Redistribution and use in source and binary forms, with or without 
modification, are permitted (subject to the limitations in the disclaimer 
below) provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, 
this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice, 
this list of conditions and the following disclaimer in the documentation 
and/or other materials provided with the distribution.
* Neither the name of InterDigital Communications, Inc nor the names of its 
contributors may be used to endorse or promote products derived from this 
software without specific prior written permission.

NO EXPRESS OR IMPLIED LICENSES TO ANY PARTY'S PATENT RIGHTS ARE GRANTED BY 
THIS LICENSE. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND 
CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT 
NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A 
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER 
OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, 
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, 
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; 
OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR 
OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF 
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. -->

# Knowledge Distillation Applied to LIC

In [ ]:
import time
import io
import math
import numpy as np

from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from pytorch_msssim import ms_ssim

from compressai.zoo.image import (
    model_urls,
    load_state_dict_from_url,
    load_pretrained,
    _load_model,
    bmshj2018_hyperprior,
    mbt2018_mean,
    mbt2018
)

from fvcore.nn import FlopCountAnalysis
from zeus.monitor import ZeusMonitor
from zeus.device.cpu import get_current_cpu_index

import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

from models import ScaleHyperprior, MeanScaleHyperprior, JointAutoregressiveHierarchicalPriors

In [ ]:
def model_nb_param(model):
    return sum(p.numel() for p in model.parameters())

def model_memory_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    return size_all_mb

In [ ]:
def compute_psnr(a, b):
    mse = torch.mean((a - b)**2).item()
    return -10 * math.log10(mse)


def compute_msssim(a, b):
    return ms_ssim(a, b, data_range=1.).item()


def compute_bpp(out_net):
    size = out_net["x_hat"].size()
    num_pixels = size[0] * size[2] * size[3]
    return sum(torch.log(likelihoods).sum() / (-math.log(2) * num_pixels)
               for likelihoods in out_net["likelihoods"].values()).item()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

## Load Models

In [ ]:
N_teacher = 128
M = 192

In [ ]:
# Teacher Model
url = model_urls["bmshj2018-hyperprior"]["mse"][5]
state_dict = load_state_dict_from_url(url, progress=False)
state_dict = load_pretrained(state_dict)
teacher = ScaleHyperprior.from_state_dict(state_dict).eval().to(device)

In [ ]:
# Student Model (16 channels)
# id = 258263
# id = 259782 # KD (MSE, MSE, MSE)
id = 263674 # KD (MSE, MSE, RD)
student_16 = ScaleHyperprior(16, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_16.load_state_dict(checkpoint["state_dict"])
student_16 = student_16.eval().to(device)

In [ ]:
# Student Model (32 channels)
# id = 258258
# id = 259783 # KD (MSE, MSE, MSE)
id = 274457 # KD (MSE, MSE, RD)
student_32 = ScaleHyperprior(32, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_32.load_state_dict(checkpoint["state_dict"])
student_32 = student_32.eval().to(device)

In [ ]:
# Student Model (64 channels)
# id = 258259
# id = 259784 # KD (MSE, MSE, MSE)
id = 274461 # KD (MSE, MSE, RD)
student_64 = ScaleHyperprior(64, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_64.load_state_dict(checkpoint["state_dict"])
student_64 = student_64.eval().to(device)

In [ ]:
# Student Model (96 channels)
# id = 258262
# id = 259785 # KD (MSE, MSE, MSE)
id = 274464 # KD (MSE, MSE, RD)
student_96 = ScaleHyperprior(96, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_96.load_state_dict(checkpoint["state_dict"])
student_96 = student_96.eval().to(device)

In [ ]:
# Student Model (112 channels)
# id = 259786 # KD (MSE, MSE, MSE)
id = 263691 # KD (MSE, MSE, RD)
student_112 = ScaleHyperprior(112, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_112.load_state_dict(checkpoint["state_dict"])
student_112 = student_112.eval().to(device)

In [ ]:
# %%script false --no-raise-error
# Student Model (112 channels)
id = 261095 # KD (MSE, MSE, RD[KL])
student_112_kl = ScaleHyperprior(112, M)
checkpoint = torch.load(f"train_res/{id}/checkpoint_best.pth.tar",
    weights_only=True, map_location=torch.device("cpu"))
student_112_kl.load_state_dict(checkpoint["state_dict"])
student_112_kl = student_112_kl.eval().to(device)

In [ ]:
models = {
    "teacher": {"model": teacher, "metrics": {"params": model_nb_param(teacher), "memory": model_memory_size(teacher)}},
    "student_16": {"model": student_16, "metrics": {"params": model_nb_param(student_16), "memory": model_memory_size(student_16)}},
    "student_32": {"model": student_32, "metrics": {"params": model_nb_param(student_32), "memory": model_memory_size(student_32)}},
    "student_64": {"model": student_64, "metrics": {"params": model_nb_param(student_64), "memory": model_memory_size(student_64)}},
    "student_96": {"model": student_96, "metrics": {"params": model_nb_param(student_96), "memory": model_memory_size(student_96)}},
    "student_112": {"model": student_112, "metrics": {"params": model_nb_param(student_112), "memory": model_memory_size(student_112)}},
    # "student_112_kl": {"model": student_112_kl, "metrics": {"params": model_nb_param(student_112_kl), "memory": model_memory_size(student_112_kl)}},
}

## Models Comparison

### Saint Malo

In [ ]:
test_transforms = transforms.Compose(
    [transforms.RandomCrop((256,256)), transforms.ToTensor()]
)

img = Image.open("../data/assets/stmalo_fracape.png").convert("RGB")
x = test_transforms(img).unsqueeze(0).to(device)
print(x.shape)

In [ ]:
# Flops computation
with torch.no_grad():
    for model in models.keys():
        flops = FlopCountAnalysis(models[model]["model"], x)
        models[model]["metrics"]["flops"] = flops.total()

In [ ]:
%%script false --no-raise-error
# Energy consumption computation
current_cpu_index = get_current_cpu_index()
monitor = ZeusMonitor(cpu_indices=[current_cpu_index], gpu_indices=[])
with torch.no_grad():
    for model in models.keys():
        monitor.begin_window("step")
        output = models[model]["model"].forward(x)
        monitor.end_window("step")

In [ ]:
# Inference
criterion = nn.MSELoss()
with torch.no_grad():
    for model in models.keys():
        start = time.time()
        output = models[model]["model"].forward(x)
        stop = time.time()

        models[model]["metrics"]["inference time"] = stop - start
        models[model]["metrics"]["mse"] = criterion(output["x_hat"], x).item()
        models[model]["metrics"]["psnr"] = compute_psnr(output["x_hat"], x)
        models[model]["metrics"]["ms_ssim"] = compute_msssim(output["x_hat"], x)
        models[model]["metrics"]["bpp"] = compute_bpp(output)

        models[model]["output"] = output

In [ ]:
n_rows = len(models)
fig, axs = plt.subplots(n_rows, 4, figsize=(20, 4*n_rows))

orig = transforms.ToPILImage()(x.squeeze().cpu())

for ax in axs.ravel():
    ax.axis("off")

for i, model in enumerate(models.keys()):
    axs[i, 0].imshow(orig)
    axs[i, 0].title.set_text("Original")

    recons = transforms.ToPILImage()(models[model]["output"]["x_hat"].squeeze().cpu())
    axs[i, 1].imshow(recons)
    axs[i, 1].title.set_text(model)

    diffs = torch.mean((models[model]["output"]["x_hat"] - x).abs(), axis=1).squeeze().cpu()
    axs[i, 2].imshow(diffs, cmap="viridis")
    axs[i, 2].title.set_text(f"Difference")

    axs[i, 3].set_axis_off()
    tab = axs[i, 3].table(rowLabels=list(models[model]["metrics"].keys()),
                          cellText=[[f"{m:,}"] if isinstance(m, int) else [f"{m:.6f}"] for m in models[model]["metrics"].values()],
                          loc="center")
    tab.auto_set_column_width(col=[0, 1])
    tab.auto_set_font_size(False)


plt.show()

## Create Figure

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
id = "20250529_180652"
with open(f"test_res/{id}/avg_metrics_kodak.json") as f:
    exp_1 = json.load(f)

In [ ]:
id = "20250526_214614"
with open(f"test_res/{id}/avg_metrics_kodak.json") as f:
    exp_2 = json.load(f)

In [ ]:
pretrained = exp_1["pretrained"]
exp_1 = exp_1["ours"]
exp_2 = exp_2["ours"]

In [ ]:
# Retrieve average metrics as lists
pretrained_brs = [m["bit-rate"] for _, m in pretrained.items()]
pretrained_psnrs = [m["psnr"] for _, m in pretrained.items()]
pretrained_msssim = [-10*np.log10(1-m["ms-ssim"]) for _, m in pretrained.items()]
brs_1 = [m["bit-rate"] for _, m in exp_1.items()]
psnrs_1 = [m["psnr"] for _, m in exp_1.items()]
msssim_1 = [-10*np.log10(1-m["ms-ssim"]) for _, m in exp_1.items()]
brs_2 = [m["bit-rate"] for _, m in exp_2.items()]
psnrs_2 = [m["psnr"] for _, m in exp_2.items()]
msssim_2 = [-10*np.log10(1-m["ms-ssim"]) for _, m in exp_2.items()]

In [ ]:
plt.rcParams["axes.prop_cycle"] = plt.rcParams["axes.prop_cycle"][1:]

# Plot average rate-distortion curves
fig, axs = plt.subplots(1, 2, figsize=(13, 5))

axs[0].plot(pretrained_brs[3:-3], pretrained_psnrs[3:-3], "blue", linestyle="--",
            linewidth=1, label="pre-trained")
axs[0].plot(brs_1[-2:], psnrs_1[-2:], "red", linestyle="--", linewidth=1,
            label=f"ours")
axs[1].plot(pretrained_brs[3:-3], pretrained_msssim[3:-3], "blue", linestyle="--",
            linewidth=1, label="pre-trained")
axs[1].plot(brs_1[-2:], msssim_1[-2:], "red", linestyle="--", linewidth=1, label="ours")

for name, m in pretrained.items():
    # if name in ["1", "2", "3", "4", "5"]:
    if name in ["4", "5"]:
        axs[0].plot(m["bit-rate"], m["psnr"], "o", color="blue")
        axs[0].grid(True)
        axs[0].set_ylabel("PSNR [dB]")
        axs[0].set_xlabel("Bit rate [bpp]")
        axs[0].title.set_text("PSNR comparison")

        axs[1].plot(m["bit-rate"], -10*np.log10(1-m["ms-ssim"]), "o", color="blue")
        axs[1].grid(True)
        axs[1].set_ylabel("MS-SSIM [dB]")
        axs[1].set_xlabel("Bit rate [bpp]")
        axs[1].title.set_text("MS-SSIM (log) comparison")

for name, m in exp_1.items():
    if name in ["teacher", "student_5", "student_4"]:
        axs[0].plot(m["bit-rate"], m["psnr"],
                    "s" if name == "teacher" else "o", label=name)
        axs[0].grid(True)
        axs[0].set_ylabel("PSNR [dB]")
        axs[0].set_xlabel("Bit rate [bpp]")
        axs[0].title.set_text("PSNR comparison")

        axs[1].plot(m["bit-rate"], -10*np.log10(1-m["ms-ssim"]),
                    "s" if name == "teacher" else "o", label=name)
        axs[1].grid(True)
        axs[1].set_ylabel("MS-SSIM [dB]")
        axs[1].set_xlabel("Bit rate [bpp]")
        axs[1].title.set_text("MS-SSIM (log) comparison")

for name, m in exp_2.items():
    if name != "teacher":
        new_name = "student_5_" + name.split("_")[1]
        axs[0].plot(m["bit-rate"], m["psnr"],
                    "s" if name == "teacher" else "o", label=new_name)
        axs[0].grid(True)
        axs[0].set_ylabel("PSNR [dB]")
        axs[0].set_xlabel("Bit rate [bpp]")
        axs[0].title.set_text("PSNR comparison")

        axs[1].plot(m["bit-rate"], -10*np.log10(1-m["ms-ssim"]),
                    "s" if name == "teacher" else "o", label=new_name)
        axs[1].grid(True)
        axs[1].set_ylabel("MS-SSIM [dB]")
        axs[1].set_xlabel("Bit rate [bpp]")
        axs[1].title.set_text("MS-SSIM (log) comparison")

axs[0].legend(loc="best")
axs[1].legend(loc="best")

fig.tight_layout()

plt.savefig(f"avg_rd_curve_kodak.png")